# Tutorial 1.7: Evaluating Agents with MLflow

## LLM-as-Judge Evaluation for GenAI Applications

This notebook teaches you how to systematically evaluate agents and LLM applications using MLflow's evaluation framework. You'll learn to use built-in judges, create custom scorers, and integrate third-party evaluation libraries like DeepEval.

### What You'll Learn

- ✅ Why agent evaluation matters
- ✅ MLflow built-in scorers (RelevanceToQuery, Correctness, Guidelines, Safety)
- ✅ Creating custom scorers with the `@scorer` decorator
- ✅ DeepEval integration for conversational evaluation
- ✅ Session-level multi-turn evaluation

### Prerequisites
- Completed notebooks 1.1-1.6
- Understanding of MLflow tracing

### Estimated Time: 25-30 minutes

---
## Step 1: Why Evaluate Agents?

Agent evaluation is critical for:

| Challenge | Solution |
|-----------|----------|
| Non-deterministic outputs | Statistical evaluation over multiple runs |
| Complex reasoning chains | Step-by-step quality assessment |
| Multi-turn conversations | Session-level coherence metrics |
| Safety and compliance | Automated guardrail checking |
| Regression detection | Baseline comparisons |

### LLM-as-Judge Pattern

Instead of brittle string matching, we use LLMs to evaluate LLM outputs:

```
Agent Output → Judge LLM → Score + Reasoning
```

MLflow provides:
1. **Built-in scorers** - Pre-configured judges for common metrics
2. **Custom scorers** - Define your own evaluation logic
3. **Third-party integrations** - DeepEval, RAGAS, and more

---
## Step 2: Environment Setup

In [ ]:
import os
import mlflow
from openai import OpenAI
from dotenv import load_dotenv

# Load environment
load_dotenv()

# Configure MLflow
mlflow.set_tracking_uri("./mlruns")
mlflow.set_experiment("11-agent-evaluation")

# Enable autologging
mlflow.openai.autolog()

# Initialize OpenAI client
client = OpenAI()

# Judge model for evaluation
JUDGE_MODEL = "openai:/gpt-4o-mini"

print("✅ Environment configured for agent evaluation")
print(f"   MLflow tracking: {mlflow.get_tracking_uri()}")
print(f"   Experiment: 11-agent-evaluation")
print(f"   Judge model: {JUDGE_MODEL}")

---
## Step 3: Create a Simple Agent for Evaluation

Let's create a simple Q&A agent that we'll evaluate.

In [ ]:
class SimpleQAAgent:
    """
    A simple Q&A agent for demonstration.
    """
    
    def __init__(self, model: str = "gpt-4o-mini"):
        self.model = model
        self.client = OpenAI()
    
    @mlflow.trace(name="qa_agent", span_type="AGENT")
    def answer(self, question: str) -> str:
        """Answer a question."""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Provide concise, accurate answers."},
                {"role": "user", "content": question}
            ],
            temperature=0.7,
            max_tokens=200
        )
        return response.choices[0].message.content

# Initialize agent
agent = SimpleQAAgent()

# Test the agent
test_response = agent.answer("What is MLflow?")
print(f"Test response: {test_response[:200]}...")

---
## Step 4: MLflow Built-in Scorers

MLflow provides pre-configured scorers for common evaluation needs:

| Scorer | Purpose | Inputs |
|--------|---------|--------|
| `RelevanceToQuery` | Is the response relevant to the question? | inputs, outputs |
| `Correctness` | Is the response factually correct? | outputs, expectations |
| `Guidelines` | Does the response follow specific guidelines? | outputs |
| `Safety` | Is the response safe and appropriate? | outputs |

In [ ]:
from mlflow.genai.scorers import (
    RelevanceToQuery,
    Correctness,
    Guidelines,
    Safety
)

# Initialize built-in scorers
relevance_scorer = RelevanceToQuery(model=JUDGE_MODEL)
correctness_scorer = Correctness(model=JUDGE_MODEL)
safety_scorer = Safety(model=JUDGE_MODEL)

# Guidelines scorer with custom guidelines
guidelines_scorer = Guidelines(
    model=JUDGE_MODEL,
    guidelines=[
        "Responses should be concise (under 100 words)",
        "Responses should not include harmful content",
        "Responses should be helpful and informative"
    ],
    name="custom_guidelines"
)

print("✅ Built-in scorers initialized:")
print("   - RelevanceToQuery")
print("   - Correctness")
print("   - Safety")
print("   - Guidelines (custom)")

---
## Step 5: Create Evaluation Dataset

An evaluation dataset contains:
- `inputs`: The questions/prompts sent to the agent
- `expectations` (optional): Expected answers for correctness checking

In [ ]:
# Evaluation dataset
eval_dataset = [
    {
        "inputs": {"question": "What is MLflow?"},
        "expectations": {"expected_answer": "MLflow is an open source platform for managing the machine learning lifecycle, including experiment tracking, model packaging, and deployment."}
    },
    {
        "inputs": {"question": "What are the main components of MLflow?"},
        "expectations": {"expected_answer": "MLflow has four main components: Tracking, Projects, Models, and Model Registry."}
    },
    {
        "inputs": {"question": "How does MLflow help with experiment tracking?"},
        "expectations": {"expected_answer": "MLflow Tracking allows you to log parameters, metrics, artifacts, and models during ML experiments, enabling comparison and reproducibility."}
    },
    {
        "inputs": {"question": "What is MLflow Tracing used for?"},
        "expectations": {"expected_answer": "MLflow Tracing provides observability for GenAI applications by capturing LLM calls, retrieval steps, and agent reasoning with full visibility."}
    },
]

print(f"✅ Evaluation dataset created with {len(eval_dataset)} examples")
for i, item in enumerate(eval_dataset, 1):
    print(f"   {i}. {item['inputs']['question'][:50]}...")

In [ ]:
def predict_fn(inputs: dict) -> dict:
    """
    Prediction function wrapper for evaluation.
    
    Args:
        inputs: Dictionary with 'question' key
        
    Returns:
        Dictionary with 'response' key
    """
    question = inputs["question"]
    response = agent.answer(question)
    return {"response": response}

print("✅ Prediction function defined")

---
## Step 6: Run Evaluation with Built-in Scorers

In [ ]:
# Run evaluation
print("🔄 Running evaluation with built-in scorers...\n")

results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=predict_fn,
    scorers=[
        relevance_scorer,
        correctness_scorer,
        safety_scorer,
        guidelines_scorer
    ]
)

print("\n✅ Evaluation complete!")
print(f"\nMetrics Summary:")
print("-" * 40)
for metric_name, value in results.metrics.items():
    if isinstance(value, float):
        print(f"  {metric_name}: {value:.3f}")
    else:
        print(f"  {metric_name}: {value}")

In [ ]:
# View detailed results
print("\n📊 Detailed Results:")
print("=" * 80)

# Display the results table
results.tables["eval_results"]

---
## Step 7: Custom Scorers with @scorer Decorator

Create your own evaluation logic using the `@scorer` decorator.

In [ ]:
from mlflow.genai import scorer
from mlflow.genai.scorers import Scorer

@scorer
def response_length_check(outputs: dict) -> bool:
    """
    Check if response is within acceptable length.
    Returns True if response is between 20 and 500 characters.
    """
    response = outputs.get("response", "")
    length = len(response)
    return 20 <= length <= 500

@scorer
def contains_keywords(outputs: dict, expectations: dict) -> bool:
    """
    Check if response contains key terms from expected answer.
    """
    response = outputs.get("response", "").lower()
    expected = expectations.get("expected_answer", "").lower()
    
    # Extract key words (simple approach)
    key_words = [word for word in expected.split() if len(word) > 4]
    
    # Check if at least 30% of key words are present
    if not key_words:
        return True
    
    matches = sum(1 for word in key_words if word in response)
    return matches / len(key_words) >= 0.3

@scorer
def no_hallucination_markers(outputs: dict) -> bool:
    """
    Check for common hallucination markers.
    """
    response = outputs.get("response", "").lower()
    
    hallucination_markers = [
        "i think",
        "i believe",
        "probably",
        "might be",
        "i'm not sure",
        "as far as i know"
    ]
    
    return not any(marker in response for marker in hallucination_markers)

print("✅ Custom scorers defined:")
print("   - response_length_check")
print("   - contains_keywords")
print("   - no_hallucination_markers")

In [ ]:
# Run evaluation with custom scorers
print("🔄 Running evaluation with custom scorers...\n")

custom_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=predict_fn,
    scorers=[
        response_length_check,
        contains_keywords,
        no_hallucination_markers
    ]
)

print("\n✅ Custom evaluation complete!")
print(f"\nCustom Metrics Summary:")
print("-" * 40)
for metric_name, value in custom_results.metrics.items():
    if isinstance(value, float):
        print(f"  {metric_name}: {value:.3f}")
    else:
        print(f"  {metric_name}: {value}")

---
## Step 8: DeepEval Integration

MLflow integrates with DeepEval for advanced conversational AI evaluation metrics.

| DeepEval Scorer | Purpose |
|-----------------|----------|
| `ConversationCompleteness` | Did the conversation achieve its goal? |
| `KnowledgeRetention` | Does the agent remember context? |
| `TopicAdherence` | Does the agent stay on topic? |

In [ ]:
from mlflow.genai.scorers.deepeval import (
    ConversationCompleteness,
    KnowledgeRetention,
    TopicAdherence
)

# Initialize DeepEval scorers
completeness_scorer = ConversationCompleteness(model=JUDGE_MODEL)
retention_scorer = KnowledgeRetention(model=JUDGE_MODEL)
topic_scorer = TopicAdherence(
    model=JUDGE_MODEL,
    allowed_topics=["MLflow", "machine learning", "AI", "data science", "experiment tracking"]
)

print("✅ DeepEval scorers initialized:")
print("   - ConversationCompleteness")
print("   - KnowledgeRetention")
print("   - TopicAdherence")

---
## Step 9: Multi-Turn Conversation Agent

For session-level evaluation, we need an agent that handles conversations.

In [ ]:
import uuid

class ConversationalAgent:
    """
    An agent that maintains conversation history for multi-turn interactions.
    """
    
    def __init__(self, model: str = "gpt-4o-mini"):
        self.model = model
        self.client = OpenAI()
        self.conversation_history = []
        self.session_id = str(uuid.uuid4())
    
    def reset(self):
        """Reset conversation history and start new session."""
        self.conversation_history = []
        self.session_id = str(uuid.uuid4())
    
    @mlflow.trace(name="conversational_agent", span_type="AGENT")
    def chat(self, user_message: str) -> str:
        """
        Send a message and get a response, maintaining history.
        """
        # Tag trace with session ID for grouping
        mlflow.update_current_trace(metadata={
            "mlflow.trace.session": self.session_id,
            "turn_number": len(self.conversation_history) // 2 + 1
        })
        
        # Add user message to history
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        # Prepare messages with system prompt
        messages = [
            {"role": "system", "content": "You are a helpful MLflow expert assistant. Answer questions about MLflow clearly and concisely. Remember previous context in the conversation."}
        ] + self.conversation_history
        
        # Get response
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.7,
            max_tokens=300
        )
        
        assistant_message = response.choices[0].message.content
        
        # Add to history
        self.conversation_history.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        return assistant_message

print("✅ ConversationalAgent defined with session tracking")

In [ ]:
# Simulate a multi-turn conversation
conv_agent = ConversationalAgent()

conversation_turns = [
    "What is MLflow?",
    "What are its main components?",
    "Tell me more about the Tracking component.",
    "How does it compare to other tools?"
]

print("🗣️ Multi-Turn Conversation\n")
print("=" * 60)

for i, user_msg in enumerate(conversation_turns, 1):
    print(f"\n[Turn {i}]")
    print(f"User: {user_msg}")
    response = conv_agent.chat(user_msg)
    print(f"Agent: {response[:200]}..." if len(response) > 200 else f"Agent: {response}")

print("\n" + "=" * 60)
print(f"\n✅ Conversation complete (Session: {conv_agent.session_id[:8]}...)")

---
## Step 10: Session-Level Evaluation with DeepEval

In [ ]:
# Search for traces from this session
session_traces = mlflow.search_traces(
    filter_string=f"metadata.`mlflow.trace.session` = '{conv_agent.session_id}'"
)

print(f"📊 Found {len(session_traces)} traces for session {conv_agent.session_id[:8]}...")

# Display trace info
for i, trace in enumerate(session_traces.itertuples(), 1):
    print(f"   Turn {i}: {trace.request_id[:8]}...")

In [ ]:
# Prepare conversation data for DeepEval evaluation
conversation_eval_data = [
    {
        "inputs": {
            "messages": conv_agent.conversation_history
        },
        "expectations": {
            "goal": "Help the user understand MLflow and its components"
        }
    }
]

# Define a predict function that returns the conversation
def conversation_predict_fn(inputs: dict) -> dict:
    """Return the conversation as-is for evaluation."""
    return {"messages": inputs["messages"]}

print("🔄 Running session-level evaluation with DeepEval scorers...\n")

deepeval_results = mlflow.genai.evaluate(
    data=conversation_eval_data,
    predict_fn=conversation_predict_fn,
    scorers=[
        completeness_scorer,
        retention_scorer,
        topic_scorer
    ]
)

print("\n✅ DeepEval session evaluation complete!")
print(f"\nDeepEval Metrics:")
print("-" * 40)
for metric_name, value in deepeval_results.metrics.items():
    if isinstance(value, float):
        print(f"  {metric_name}: {value:.3f}")
    else:
        print(f"  {metric_name}: {value}")

---
## Step 11: Viewing Evaluation Results in MLflow UI

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║         Viewing Evaluation Results in MLflow UI              ║
╚══════════════════════════════════════════════════════════════╝

🔍 EXPERIMENTS VIEW:
   Navigate to: http://localhost:5000
   Select: "11-agent-evaluation" experiment
   
   You'll see:
   - Evaluation runs with scorer metrics
   - Pass/fail rates for each scorer
   - Detailed reasoning from LLM judges

📊 EVALUATION RESULTS:
   Each run includes:
   - Metric values (0-1 scores or boolean)
   - Judge reasoning explanations
   - Input/output pairs
   - Artifacts with detailed results

🎯 KEY METRICS TO MONITOR:
   
   Built-in Scorers:
   - relevance_to_query/score: Response relevance
   - correctness/score: Factual accuracy
   - safety/score: Safety compliance
   - guidelines/score: Guideline adherence
   
   Custom Scorers:
   - response_length_check: Length validation
   - contains_keywords: Keyword presence
   - no_hallucination_markers: Confidence check
   
   DeepEval Scorers:
   - conversation_completeness/score: Goal achievement
   - knowledge_retention/score: Context memory
   - topic_adherence/score: Topic focus

💡 TIPS:
   1. Compare multiple evaluation runs
   2. Filter by scorer to find failures
   3. Read judge reasoning for insights
   4. Track metrics over time for regression detection

""")

---
## Summary

In this notebook, you learned how to evaluate agents and LLM applications using MLflow's evaluation framework.

### ✅ What You Learned

**Built-in Scorers:**
- `RelevanceToQuery` - Check if responses are relevant
- `Correctness` - Verify factual accuracy
- `Guidelines` - Enforce custom guidelines
- `Safety` - Ensure safe outputs

**Custom Scorers:**
- Use `@scorer` decorator for custom logic
- Access `outputs` and `expectations` in scorers
- Return boolean or numeric scores

**DeepEval Integration:**
- `ConversationCompleteness` - Goal achievement
- `KnowledgeRetention` - Context memory
- `TopicAdherence` - Topic focus

**Session-Level Evaluation:**
- Tag traces with session IDs
- Search traces by session
- Evaluate multi-turn conversations

### 🔑 Key Patterns

```python
# Built-in scorer
from mlflow.genai.scorers import RelevanceToQuery
scorer = RelevanceToQuery(model="openai:/gpt-4o-mini")

# Custom scorer
from mlflow.genai import scorer
@scorer
def my_scorer(outputs) -> bool:
    return condition

# Run evaluation
results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=predict_fn,
    scorers=[scorer1, scorer2]
)
```

### 📚 Next Steps

Continue to **Tutorial 1.8: Complete RAG Application** to build a full RAG system with RAGAS evaluation metrics.